# Setup

In [ ]:
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from sklearn.metrics import *
import pickle

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [ ]:
df = pd.read_csv(f'data/data_total.csv', index_col=0)

In [ ]:
ycol = [
    df.columns[df.columns.str.contains('wait')],
    df.columns[df.columns.str.contains('timeloss')],
    df.columns[df.columns.str.contains('travel')],
    df.columns[df.columns.str.contains('speed')],
    df.columns[df.columns.str.contains('stop')],
]

In [ ]:
y = [
    df[c].mean(1) for c in ycol
]

In [ ]:
p = pd.read_csv('preprocessed/preprocessed_p.csv', index_col=0)
q = pd.read_csv('preprocessed/preprocessed_q.csv', index_col=0)
var = pd.read_csv('preprocessed/preprocessed_var.csv', index_col=0)

In [ ]:
p.columns

In [ ]:
q.columns

In [ ]:
var.columns

In [ ]:
strat = df['id']

In [ ]:
X = pd.concat([var, p, q], 1)
X_wo_p = pd.concat([var, q], 1)
X_wo_q = pd.concat([var, p], 1)
X_wo_pq = pd.concat([var], 1)

In [ ]:
y = np.stack(y, 1)

In [ ]:
y[:,:3] /= 300
y[:, 3] = (y[:, 3]-3)/6
y[:, 4] /= 3

In [ ]:
y1 = y[:,:1]

# Load models

In [ ]:
def get_MMoE_model(num_experts=3, num_tasks=5, emb_dim=64):
    in_dim = X.shape[1]
    i_ = Input((X.shape[1], ))
    
    experts = []
    for _ in range(num_experts):
        exp = build_layer(in_dim, (256, 128), emb_dim)
        experts.append(exp(i_))
        
    expert_concat = tf.keras.layers.Lambda(lambda x: tf.stack(x, axis=1))(experts)
    
    mmoe_outs = []
    for _ in range(num_tasks):
        g_layer = build_layer(in_dim, (128, ), emb_dim)
        g_in = g_layer(i_)
        g_out = tf.keras.layers.Dense(num_experts, use_bias=False, activation='softmax')(g_in)
        g_out = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(g_out)
        
        g_mul_out = tf.keras.layers.Lambda(lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=False))([expert_concat, g_out])
        
        mmoe_outs.append(g_mul_out)
        
    task_outs = []
    for mmoe_out in mmoe_outs:
        out_layer = build_layer(emb_dim, (64, ), 1)
        out = out_layer(mmoe_out)
        task_outs.append(out)
        
    task_outs = tf.concat(task_outs, 1)
    return tf.keras.models.Model(i_, task_outs)


def build_layer(in_dim, h_dim, out_dim):
    i_ = Input((in_dim, ))
    h = i_
    for d in h_dim:
       h = Dense(d, activation='swish')(h)
    
    o = Dense(out_dim)(h)
    model = tf.keras.models.Model(i_, o)
    return model

In [ ]:
poly_reg = pickle.load(open(f'models/model_poly_reg/model.pkl', "rb"))

simple_MLP_y1_model = tf.keras.models.load_model(f'models/model_simple_MLP_y1')

simple_MLP_y5_model = tf.keras.models.load_model(f'models/model_simple_MLP_y5')

shared_bottom_model = tf.keras.models.load_model(f'models/model_shared_bottom')

OMoE_model = tf.keras.models.load_model(f'models/model_OMoE')
MMoE_model = tf.keras.models.load_model(f'models/model_MMoE_original')

MMoE_wo_p_model = tf.keras.models.load_model(f'models/model_MMoE_wo_traffic_param')
MMoE_wo_q_model = tf.keras.models.load_model(f'models/model_MMoE_wo_network_param')
MMoE_wo_pq_model = tf.keras.models.load_model(f'models/model_MMoE_wo_both_param')

In [ ]:
models = {
    'poly_reg': poly_reg,
    'simple_MLP_y1_model': simple_MLP_y1_model,
    'simple_MLP_y5_model': simple_MLP_y5_model,
    'shared_bottom_model': shared_bottom_model,
    'OMoE_model': OMoE_model,
    'MMoE_model': MMoE_model,
    'MMoE_wo_p_model': MMoE_wo_p_model,
    'MMoE_wo_q_model': MMoE_wo_q_model,
    'MMoE_wo_pq_model': MMoE_wo_pq_model
}

# Predict test sets

In [ ]:
import csv

with open('data/test_networks.csv', 'r', newline='') as myfile:
     wr = csv.reader(myfile, quoting=csv.QUOTE_ALL)
     test_data = list(wr)[0]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer

In [ ]:
from tqdm import tqdm

val_id = list(tqdm(test_data))

In [ ]:
tr_X = X[~strat.isin(val_id)]
tr_X_wo_p = X_wo_p[~strat.isin(val_id)] 
tr_X_wo_q = X_wo_q[~strat.isin(val_id)] 
tr_X_wo_pq = X_wo_pq[~strat.isin(val_id)] 
tr_y = y[~strat.isin(val_id)]
tr_y1 = y1[~strat.isin(val_id)]

val_X = X[strat.isin(val_id)]
val_X_wo_p = X_wo_p[strat.isin(val_id)]
val_X_wo_q = X_wo_q[strat.isin(val_id)]
val_X_wo_pq = X_wo_pq[strat.isin(val_id)]
val_y = y[strat.isin(val_id)]
val_y1 = y1[strat.isin(val_id)]

In [ ]:
poly = ColumnTransformer(
    transformers=[
        ('transformer', PolynomialFeatures(degree=2, include_bias=False), ['p1', 'p2', 'p3', 'p4', 'total_len']),
    ],
    remainder='passthrough'
)

tr_X_poly = poly.fit_transform(tr_X)
val_X_poly = poly.fit_transform(val_X)

## Inference on test set

In [ ]:
prediction_results = []

for model_name, model in models.items():
    num_tasks = 1 if model_name in ['poly_reg', 'simple_MLP_y1_model'] else 5
    
    if model_name == 'MMoE_wo_p_model':
        test_X = val_X_wo_p 
    elif model_name == 'MMoE_wo_q_model':
        test_X = val_X_wo_q 
    elif model_name == 'MMoE_wo_pq_model':
        test_X = val_X_wo_pq 
    elif model_name == 'poly_reg':
        test_X = val_X_poly
    else:
        test_X = val_X

    test_y = val_y1 if model_name in ['poly_reg', 'simple_MLP_y1_model'] else val_y
    
    pred = model.predict(test_X)

    trues = test_y
    
    # score writer
    tmp_scr = [model_name]
    tmp_scr.append((np.abs(pred - trues) / trues).mean() * 100)
    
    for i in range(num_tasks):
        s_ = ((np.abs(pred[:,i] - trues[:,i])*300) / (trues[:,i]*300)).mean() * 100
        tmp_scr.append(s_)

    prediction_results.append(tmp_scr)

In [ ]:
df_prediction = pd.DataFrame(prediction_results, columns = ['model', 'total', 'wait', 'timeloss', 'travel', 'speed', 'stop'])
df_prediction

# Inference on train set

In [ ]:
prediction_results = []

for model_name, model in models.items():
    num_tasks = 1 if model_name in ['poly_reg', 'simple_MLP_y1_model'] else 5
    
    if model_name == 'MMoE_wo_p_model':
        test_X = tr_X_wo_p 
    elif model_name == 'MMoE_wo_q_model':
        test_X = tr_X_wo_q 
    elif model_name == 'MMoE_wo_pq_model':
        test_X = tr_X_wo_pq 
    elif model_name == 'poly_reg':
        test_X = tr_X_poly
    else:
        test_X = tr_X

    test_y = tr_y1 if model_name in ['poly_reg', 'simple_MLP_y1_model'] else tr_y
    
    pred = model.predict(test_X)

    trues = test_y
    
    # score writer
    tmp_scr = [model_name]
    tmp_scr.append((np.abs(pred - trues) / trues).mean() * 100)
    
    for i in range(num_tasks):
        s_ = ((np.abs(pred[:,i] - trues[:,i])*300) / (trues[:,i]*300)).mean() * 100
        tmp_scr.append(s_)

    prediction_results.append(tmp_scr)

In [ ]:
df_prediction_train = pd.DataFrame(prediction_results, columns = ['model', 'total', 'wait', 'timeloss', 'travel', 'speed', 'stop'])
df_prediction_train